## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-25-03-54-59 +0000,nyt,"At a Congressional Hearing, Residents Detail t...",https://www.nytimes.com/2025/11/24/us/ice-raid...
1,2025-11-25-03-52-00 +0000,wsj,KKR Consortium Sold Luxury Hyatt Hotel in Toky...,https://www.wsj.com/business/deals/kkr-consort...
2,2025-11-25-03-44-43 +0000,nypost,"NYPD to hand out over 3,000 turkeys to needy f...",https://nypost.com/2025/11/24/us-news/nypd-han...
3,2025-11-25-03-41-29 +0000,nyt,Trump Moves Toward Labeling Parts of Muslim Br...,https://www.nytimes.com/2025/11/24/us/politics...
4,2025-11-25-03-40-44 +0000,bbc,Australia senator suspended after burka stunt ...,https://www.bbc.com/news/articles/cz94pdkzqvwo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
22,trump,43
53,ukraine,23
55,peace,19
56,plan,18
54,russia,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
151,2025-11-24-20-17-58 +0000,nypost,"US, Ukraine agree on new 19-point peace plan t...",https://nypost.com/2025/11/24/world-news/us-uk...,135
296,2025-11-24-07-31-06 +0000,nyt,"Ukraine, U.S. Meet About Trump’s Peace Plan to...",https://www.nytimes.com/2025/11/23/world/europ...,128
47,2025-11-25-00-42-05 +0000,nyt,What to Know About Trump’s Peace Plan for Russ...,https://www.nytimes.com/2025/11/24/us/politics...,117
7,2025-11-25-03-24-31 +0000,nypost,Initial Ukraine-Russia peace plan was hashed o...,https://nypost.com/2025/11/24/us-news/initial-...,115
95,2025-11-24-22-43-00 +0000,wsj,Ukraine Peace-Plan Negotiators Leave Thorniest...,https://www.wsj.com/world/europe/big-gaps-stil...,111


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
151,135,2025-11-24-20-17-58 +0000,nypost,"US, Ukraine agree on new 19-point peace plan t...",https://nypost.com/2025/11/24/world-news/us-uk...
187,59,2025-11-24-17-49-48 +0000,nypost,Judge dismisses cases against ex-FBI Director ...,https://nypost.com/2025/11/24/us-news/judge-di...
175,56,2025-11-24-18-33-00 +0000,wsj,President Trump’s advisers are discussing a pr...,https://www.wsj.com/politics/policy/trump-advi...
7,55,2025-11-25-03-24-31 +0000,nypost,Initial Ukraine-Russia peace plan was hashed o...,https://nypost.com/2025/11/24/us-news/initial-...
23,40,2025-11-25-02-00-00 +0000,wsj,Allies of Federal Reserve Chair Jerome Powell ...,https://www.wsj.com/economy/central-banking/fe...
41,34,2025-11-25-01-03-37 +0000,nypost,NYC DSA leaders get coveted roles in Zohran Ma...,https://nypost.com/2025/11/24/us-news/nyc-dsa-...
102,33,2025-11-24-22-22-05 +0000,wapo,"In Gulf of America case, AP renews legal fight...",https://www.washingtonpost.com/business/2025/1...
161,31,2025-11-24-19-33-58 +0000,bbc,Pentagon to review Senator Mark Kelly miscondu...,https://www.bbc.com/news/articles/cqxqwrlp7r0o...
16,30,2025-11-25-02-49-33 +0000,nypost,Unemployed son dressed as dead mom to collect ...,https://nypost.com/2025/11/24/world-news/unemp...
113,28,2025-11-24-21-59-32 +0000,nypost,Trump accepts Xi’s invite to Beijing and calls...,https://nypost.com/2025/11/24/us-news/trump-ac...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
